In [313]:
from pymongo import MongoClient
import urllib.parse
from openpyxl import load_workbook
import os
import sys

In [315]:
username = urllib.parse.quote_plus('root')
password = urllib.parse.quote_plus('schooltime')

client = MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client.brik
collection = db.tasks
db.tasks.drop()

In [316]:
data_path = 'test-data'
data_files = os.listdir(data_path)
for data_file in data_files:
    tasks = extract_with_preprocess(os.path.join(data_path, data_file))
    collection.insert_many(tasks)

In [51]:
def extract_tasks(data_file):
    wb = load_workbook(filename=data_file, read_only=True)
    ws = wb.active
    tasks = []
    total_tasks = 0
    in_chapter = False
    last_lesson = None
    last_chapter = None
    for row in ws.rows:
        if "Grade" in repr(row[0].value):
            total_tasks += int(row[0].value.split(':')[2])
        if "Chapter" in repr(row[0].value):
            if "Test" not in row[0].value:
                last_chapter = row[0].value
                in_chapter = True
                continue
        if in_chapter:
            if row[0].value == "Worksheet":
                continue
            if "Lesson:" in repr(row[0].value):
                last_lesson = row[0].value
                continue
            elif "Week of:" in repr(row[0].value):
                in_chapter = False
            else:
                tasks.append({"task":row[0].value,
                              "lesson":last_lesson,
                              "chapter":last_chapter})
    wb.close()
    return tasks

def extract_tasks_by_week(data_file):
    wb = load_workbook(filename=data_file, read_only=True)
    ws = wb.active
    tasks = []
    working_rows = ws.rows
    working_rows_values = list(map(lambda x: x[0].value, working_rows))
    working_rows_values = list(filter(lambda y: y != None, working_rows_values))
    weeks = []
    week_boundries = find_value_bounds("Week", working_rows_values)
    for i in range(len(week_boundries) - 1):
        weeks.append(working_rows_values[week_boundries[i]+1:week_boundries[i+1]])
    for week in weeks:
        subject = week[0]
        grade = week[1].split("|")[0].strip()
        chapter = week[2]
        current_lesson = ""
        for item in week[3:]:
            if "Lesson:" in item:
                current_lesson = item
                continue
            else:
                tasks.append({
                "subject": subject,
                "lesson": current_lesson,
                "grade": grade,
                "chapter": chapter,
                "task": item
                })
    return tasks

def find_value_bounds(value, array):
    bounds = []
    for i in range(len(array)):
        if value in array[i]:
            bounds.append(i)
    return bounds

def split_by_bounds(bounds, array):
    split_arrays = []
    for i in range(len(bounds) - 1):
        split_arrays.append(array[bounds[i]:bounds[i+1]])
    split_arrays.append(array[bounds[-1]:])
    return split_arrays

def extract_with_preprocess(data_file):
    wb = load_workbook(filename=data_file, read_only=True)
    ws = wb.active
    tasks = []
    # get first collumn of each row
    working_rows = ws.rows
    working_rows_values = list(map(lambda x: x[0].value, working_rows))
    # remove all null values
    working_rows_values = list(filter(lambda y: y != None, working_rows_values))
    # extract repeated info then delete
    subject = working_rows_values[1]
    grade = working_rows_values[2].split("|")[0].strip()
    working_rows_values = list(filter(lambda y: y != subject and grade not in y and "Week of:" not in y, working_rows_values))
    # find all chapter bounds
    chapter_bounds = find_value_bounds('Chapter', working_rows_values)
    chapters = split_by_bounds(chapter_bounds, working_rows_values)
    # extract tasks with lesson info
    order = 0
    for chapter in chapters:
        current_lesson = ""
        for item in chapter[1:]:
            if "Lesson:" in item:
                current_lesson = item
                continue
            else:
                tasks.append({
                "subject": subject,
                "lesson": current_lesson,
                "grade": grade,
                "chapter": chapter[0],
                "task": item,
                "completed": False,
                "weight": 3 if ('lab' in item.lower()) else 1,
                "order": order
                })
                order += 1
    return tasks

In [156]:
# how to schedule
# see how many classes need to be done
# see how many days each class has
# see how many tasks each day can have
# get start date

# differnt ways to schedule
# have a target start and end date and evenly distribute tasks 
# have a start and ideal task density then calculate how long it will take
db.week.drop()
week = {
        'monday': {'max_tasks':12},
        'tuesday': {'max_tasks':12},
        'wednesday': {'max_tasks':12},   
        'thursday': {'max_tasks':12},
        'friday': {'max_tasks':12},   
        'saturday': {'max_tasks':12},   
        'sunday': {'max_tasks':0}    
        }

for day in week:
    db.week.insert_one({day : week[day]})


In [319]:
import datetime
today = datetime.date.today()
current_day = 0
daily_tasks = []
current_weight = 0
science_tasks = collection.find({'subject':'MS Physical Science'})
lw = list(week)
while True:
    try:
        task = science_tasks.next()
        daily_tasks.append(task)
        current_weight += task['weight']
        if current_weight > week[lw[current_day%7]]['max_tasks']:
            current_day += 1
            current_weight = 0
        daily_tasks[-1]['scheduled_for'] = today + datetime.timedelta(days=current_day)
    except:
        break
    

In [318]:
for task in daily_tasks:
    query = {'_id':task['_id']}
    update = {"$set":{'scheduled_for':datetime.datetime.combine(task['scheduled_for'],datetime.time.min)}}
    db.tasks.update_one(query,update)

In [299]:
this_week = db.tasks.find({'scheduled_for': { '$lt': datetime.datetime.today() + datetime.timedelta(days=7)}})

In [308]:
this_week = this_week.rewind()
this_week.retrieved

0

In [307]:
x = 0
for x in range(this_week.retrieved - 1):
    task = this_week.next()
    print(f"{task['subject']}, {task['lesson']}, {task['chapter']}, {task['scheduled_for']}")

In [297]:
task

{'_id': ObjectId('62e58ae1a05656d12a32dc56'),
 'subject': 'MS Physical Science',
 'lesson': 'Lesson: Chemical Bonding',
 'grade': 'Middle School',
 'chapter': 'Chapter 3: Bonding',
 'task': 'Assignment: Describe chemical bonds.',
 'completed': False,
 'weight': 1,
 'order': 84,
 'scheduled_for': datetime.datetime(2022, 8, 6, 0, 0)}